In [ ]:
import zea
import matplotlib.pyplot as plt
import jax
import numpy as np
import ulsa.ops
from tqdm import tqdm
from ulsa.pipeline import beamforming

zea.init_device()

In [ ]:
acq = "20251222_s3_a4ch_line_dw_0000"

In [ ]:
path = f"/mnt/z/usbmd/Wessel/Verasonics/2026_USBMD_A4CH_S51/{acq}.hdf5"

file = zea.File(path)
frame_nr = -1
scan = file.scan()
from ulsa.utils import update_scan_for_polar_grid

scan.set_transmits("focused")
update_scan_for_polar_grid(
    scan,
    ray_multiplier=6,
    harmonic_imaging=True,
    f_number=0.3,
    pfield_kwargs={
        "downsample": 1,
        "downmix": 1,
        "percentile": 1,
        "alpha": 0.5,
        "norm": False,
    },
)
# Avoid the first pixel being at r = 0
# dr = scan.zlims[1] - scan.zlims[0] / scan.grid_size_z
# scan.zlims = (scan.zlims[0] + dr, scan.zlims[1] + dr)

width = 2e6  # Hz
f1 = scan.demodulation_frequency - width / 2
f2 = scan.demodulation_frequency + width / 2
bpf = zea.func.get_band_pass_filter(128, scan.sampling_frequency, f1, f2)

raw_data = file.load_data("raw_data", (frame_nr, scan.selected_transmits))

In [ ]:
attenuation_coef = 0.3  # dB/cm/MHz
tgc_curve = zea.func.make_tgc_curve(
    scan.n_ax,
    attenuation_coef,
    scan.sampling_frequency,
    scan.center_frequency,
    scan.sound_speed,
)
tgc_fn = lambda x: x * tgc_curve
# scan.distance_to_apex = 0.0
rx_apo = ulsa.ops.lines_rx_apo(
    n_tx=len(scan.selected_transmits), grid_size_z=scan.grid_size_z, grid_size_x=scan.grid_size_x
)

pipeline = zea.Pipeline(beamforming(rx_apo=True), with_batch_dim=False)
pipeline.append(zea.ops.ScanConvert(order=1))

params = pipeline.prepare_parameters(scan=scan, bandpass_rf=bpf, bandwidth=2e6, rx_apo=rx_apo)
output = pipeline(data=raw_data, **params)
data = output["data"]

plt.figure(figsize=(10, 5))
plt.imshow(
    data,
    cmap="gray",
    vmin=-60,
    vmax=0,
    interpolation="nearest",
    extent=scan.extent * 1e3,
)
plt.savefig("processed_image.png", dpi=300, bbox_inches="tight")

In [ ]:
def get_first_frame(file, scan):
    raw_data_frames = file.load_data("raw_data", (slice(None), scan.selected_transmits))

    pipeline = zea.Pipeline(beamforming(rx_apo=True), with_batch_dim=False)
    pipeline.append(zea.ops.keras_ops.ExpandDims(axis=-1))
    pipeline.append(zea.ops.LeeFilter(sigma=5))
    pipeline.append(zea.ops.keras_ops.CropImages(top_cropping=50))
    params = pipeline.prepare_parameters(
        scan=scan, rx_apo=rx_apo, bandpass_rf=bpf, bandwidth=2e6
    )

    maxval = None
    data_list = []
    for raw_data in tqdm(raw_data_frames):
        output = pipeline(data=raw_data, maxval=maxval, **params)
        maxval = output["maxval"]
        data = output["data"]
        data = zea.func.translate(data, (-60, 0), (0, 1))
        data = np.array(data)
        data = np.clip(data, 0, 1)
        data_list.append(data.squeeze())
    data_array = np.stack(data_list, axis=0)
    diff = np.abs(np.diff(data_array, axis=0)).mean(axis=(1, 2))
    return diff.argmax(), diff


get_first_frame(file, scan)

In [ ]:
from tqdm import tqdm


def raw_to_mp4(
    file: zea.File,
    video_path,
    pipeline: zea.Pipeline,
    scan: zea.Scan,
    frames=None,
    **kwargs,
):
    assert pipeline.with_batch_dim is False, "Pipeline must be without batch dimension"
    if frames is None:
        frames = slice(None)
    raw_data_frames = file.load_data("raw_data", (frames, scan.selected_transmits))
    data_list = []
    params = pipeline.prepare_parameters(scan=scan, **kwargs)
    for raw_data in tqdm(raw_data_frames):
        data = pipeline(data=raw_data, **params)["data"]
        data = zea.display.to_8bit(data, (-60, 0), pillow=False)
        data_list.append(data)
    zea.io_lib.save_video(
        jax.numpy.stack(data_list), video_path, fps=scan.frames_per_second
    )
    return data_list


name = file.stem + "_zea.mp4"
raw_to_mp4(file, file.path.parent / name, pipeline, scan, bpf=bpf, bandwidth=2e6)

In [ ]:
plt.figure(figsize=(10, 5))

if "non_standard_elements/verasonics_image_buffer" in file:
    vera_image = file["non_standard_elements/verasonics_image_buffer"][frame_nr][0].T
else:
    vera_image = file.load_data("image", frame_nr)
plt.imshow(vera_image, cmap="gray", vmin=-60, vmax=0)
# plt.savefig("vera_image.png", dpi=300)

In [ ]:
vera_images = file.load_data("image")
vera_images = np.clip(vera_images, a_min=-60, a_max=0)
vera_images = zea.display.to_8bit(vera_images, (-60, 0), pillow=False)
name = file.stem + "_vera.mp4"
zea.io_lib.save_video(vera_images, file.path.parent / name)